## Assignment 11
#### By Osmond Oke

In [1]:
import sys
import random
from keras import layers
import keras
import numpy as np

In [2]:
path = keras.utils.get_file('nietzsche.txt',
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower().replace('\n', '')
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 590959


In [3]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 196967
Unique characters: 56
Vectorization...


In [4]:
#building the network
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
#function to sample the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
for epoch in range(1, 20):
    print('\nepoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    model.save(f'LTSM_Model_{epoch}.h5')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
        writer.write(f'GENERATED TEXT FOR EPOCH {epoch}')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n------ temperature:', temperature)

        with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
            writer.write('\n-------------------\nTEMPERATURE: ' + str(temperature) + '\n')
            writer.write((generated_text).upper() + '\n\n')

            sys.stdout.write(generated_text.upper())

            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                generated_text += next_char
                generated_text = generated_text[1:]

                sys.stdout.write(next_char)
                writer.write(next_char)



epoch 1
1539/1539 [==============================] - 200s 129ms/step - loss: 2.2758
--- Generating with seed: "tulate. theprimordial belief of all organisms is, perhaps, t"

------ temperature: 0.2
TULATE. THEPRIMORDIAL BELIEF OF ALL ORGANISMS IS, PERHAPS, The master of the stranger of the stranged the master of the still the man the strengers of the the strest of the stranger of the strenger of the who has the strengers of the strange the was the most the the strest of the strenge of the stranged that the has the strange of the strenge of the all the strenge of the stranger of the stranger of the strenger of the self-and the conscience the stranger 
------ temperature: 0.5
OF THE STRENGER OF THE SELF-AND THE CONSCIENCE THE STRANGER of the man sattrong the groes rebled which the man still being and an all the certain and the mastentor mentic of stare the man for the structerly the menion of the fart the glace the strange the structed to how the some belong of the is for an a the truth

In [8]:
model.save('LTSM_Model.h5')